In [1]:
# making vectors

In [2]:
import pandas as pd
import numpy as np

In [9]:
df = pd.read_csv(r"C:\Users\Ekaansh\OneDrive\Desktop\AB\vs code\JS\projects\hackathon\amazon\data\train_processed.csv")


In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import re

# Combine text features
df["text_all"] = (
    df["item_name"].fillna("") + " " +
    df["bullet_points"].fillna("") + " " +
    df["product_description"].fillna("")
)

# Basic text cleaning
def clean_text(text):
    text = re.sub(r'[^a-zA-Z0-9 ]', ' ', text)  # remove special chars
    text = re.sub(r'\s+', ' ', text)             # normalize spaces
    return text.lower().strip()

df["text_all"] = df["text_all"].apply(clean_text)


In [11]:
tfidf = TfidfVectorizer(
    max_features=5000,
    stop_words='english',
    ngram_range=(1, 2)  # include unigrams and bigrams
)

X_tfidf = tfidf.fit_transform(df["text_all"])
print("TF-IDF shape:", X_tfidf.shape)


TF-IDF shape: (75000, 5000)


In [13]:
from sklearn.preprocessing import OneHotEncoder
from scipy.sparse import hstack

# Select numeric features
numeric_features = ["quantity_numeric", "price_per_unit", "value"]
X_num = df[numeric_features].fillna(0)

# One-hot encode unit_standard
ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=True)
X_cat = ohe.fit_transform(df[["unit_standard"]])

# Combine all (TF-IDF + numeric + categorical)
from scipy.sparse import csr_matrix
X_final = hstack([csr_matrix(X_num.values), X_cat, X_tfidf])

y = df["price"]

print("Final feature shape:", X_final.shape)


Final feature shape: (75000, 5016)
